# Cleaning Notebook

## 1.0 Importing Necessary Libraries

In [ ]:
# for getting data
import os
import zipfile

# for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

## 2.0 Data Cleaning

### 2.1 Crashes

2.1.1 Preview Data: .head()

2.1.2 Understand Structure: .info()

2.1.3 Format Feature names and Row Values: .lower()

2.1.4 Drop features with overly high null values: .isna().sum()/ len(df) for percentage of nulls for each feature

2.1.5 Check for duplicates: .duplicated().sum()

2.1.6 Keep or drop features with remaining nulls:

2.1.7 Inspect remaining features: .value_counts()

2.1.8 Remove unuseful features: .drop() for list of features deemed not useful for analysis; store trimmed df as ‘df_name_cleaned’

2.1.9 Reduce feature cardinality with bucketing:

2.1.10 Convert data types: stored data types to reflect true data types (categorical variables as strings, numeric variables as int, etc.)

2.1.11 Remove remaining nulls: .dropna()

Steps:
* **Preview Data**: `.head()`


* **Understand Structure**: `.info()`


* **Format Feature names and Row Values**: `.lower()`


* **Drop features with overly high null values**: `.isna().sum()/ len(df)` for percentage of nulls for each feature


* **Check for duplicates**: `.duplicated().sum()`


* **Keep or drop features with remaining nulls**:

    * make intentional decisions to keep or drop using `.value_counts()` distribution and domain knowledge


* **inspect remaining features**: `.value_counts()`; 

    * make intentional decisions to keep or drop using `.value_counts()` distribution and domain knowledge; 
    * make note of any features to keep that will need cleaning/cardinality reduction/etc.


* **remove unuseful features**: 

    * `.drop()` for list of features deemed not useful for analysis; 
    * store trimmed df as 'df_name_cleaned'
    

* **reduce feature cardinality with bucketing**:

    * `trafficway_type` and `lane_cnt`
    * `crash_hour`, `crash_day_of_week`, `crash_month`
    * `posted_speed_limit`
    * `traffic_control_device`
    * `prim_contributory_cause`
    * `most_severe_injury`
 
 
* **Convert data types**: 

    * stored data types to reflect true data types 
    * (text variables as strings, numeric variables as int, categorical as category, ect.)
    
 
* **remove remaining nulls**: `.dropna()`

In [ ]:
crashes = pd.read_csv('./data/traffic_crashes.csv', low_memory = True)
crashes.head()

In [ ]:
crashes.info()

In [ ]:
crashes.columns = crashes.columns.str.lower()

In [ ]:
# Convert all string values in object columns to lowercase
for col in crashes.select_dtypes(include='object').columns:
    crashes[col] = crashes[col].str.lower()

In [ ]:
missing_percentage = round((crashes.isna().sum()/len(crashes)*100), 2)
missing_percentage

In [ ]:
# selecting all features with 90% or more of its values are null
high_null_features = crashes.columns[(crashes.isna().sum() / len(crashes) * 100) >= 90]
high_null_features

# creating a list of features with 90% or more null values
high_null_features_list = list(high_null_features)
high_null_features_list

crashes_cleaned = crashes.drop(columns=high_null_features_list)


In [ ]:
# selecting all features with 90% or more of its values are null
high_null_features = crashes.columns[(crashes.isna().sum() / len(crashes) * 100) >= 90]
high_null_features

In [ ]:
# creating a list of features with 90% or more null values
high_null_features_list = list(high_null_features)
high_null_features_list

In [ ]:
crashes_cleaned = crashes.drop(columns=high_null_features_list)

In [ ]:
# Check for duplicate crash_record_id values in crashes_cleaned
print(f"Number of duplicate crash_record_id values in crashes_cleaned: {crashes_cleaned['crash_record_id'].duplicated().sum()}")

In [ ]:
medium_null_features = crashes.columns[
    ((crashes.isna().sum() / len(crashes) * 100) >= 60) &
    ((crashes.isna().sum() / len(crashes) * 100) < 90)
]

In [ ]:
medium_null_features_list = list(medium_null_features)

In [ ]:
for feature in medium_null_features_list:
    print(f"Value counts for column '{feature}':")
    print(crashes_cleaned[feature].value_counts())
    print("-"* 32)

hit_and_run_i is an aftermath, not a contributor so we can remove that

lane_cnt could be important. Will need to reduce cardinality though.

intersection_related_i... could be helpful. A little vague and subjective according to CDOT description. "A field observation by the police officer whether an intersection played a role in the crash. Does not represent whether or not the crash occurred within the intersection."

In [ ]:
crashes_cleaned['latitude'].value_counts()

In [ ]:
crashes_cleaned['longitude'].value_counts()

In [ ]:
crashes_cleaned['location'].value_counts()

While location data would be very helpful, the cardinality is simply too high and will restrict my limited computing power. I'll remove lat, long, and location. I'll see 

In [ ]:
crashes_cleaned['posted_speed_limit'].value_counts()

This feature could be important, but will require cardinality reduction

In [ ]:
# Categorize speed limits directly without using a function
crashes_cleaned['speed_limit_category'] = pd.cut(
    crashes_cleaned['posted_speed_limit'],
    bins=[-float('inf'), 25, 40, float('inf')],
    labels=['Low', 'Medium', 'High'],
    right=True
)

# Check the result
crashes_cleaned['speed_limit_category'].value_counts()

In [ ]:
crashes_cleaned['traffic_control_device'].value_counts()

Could be important but will need to reduce the cardinality

In [ ]:
# Create a dictionary to map the original 'traffic_control_device' values to more specific categories
traffic_control_mapping = {
    'traffic signal': 'Signal',
    'flashing control signal': 'Signal',
    'pedestrian crossing sign': 'Signal',  # If it's a signal
    'railroad crossing gate': 'Signal',    # If it uses lights
    
    'stop sign/flasher': 'Sign',
    'yield': 'Sign',
    'school zone': 'Sign',
    'railroad crossing sign': 'Sign',      # If static sign
    'other warning sign': 'Sign',
    'bicycle crossing sign': 'Sign',
    'no passing': 'Sign',
    
    'lane use marking': 'Markings & Lanes',
    'delineators': 'Markings & Lanes',
    
    'no controls': 'Other',
    'unknown': 'Other',
    'other': 'Other',
    'police/flagman': 'Other',
    'other railroad crossing': 'Other'
}

# Apply the mapping to the 'traffic_control_device' column
crashes_cleaned['traffic_control_category'] = crashes_cleaned['traffic_control_device'].map(traffic_control_mapping)

# Check the value counts for the new grouped categories
crashes_cleaned['traffic_control_category'].value_counts()

In [ ]:
crashes_cleaned['device_condition'].value_counts()

The top two categories that make up the majority of this feature are no controls and functioning properly. And then the next two frequent are unknown and other. We can drop this. won't be useful for analysis

In [ ]:
crashes_cleaned['weather_condition'].value_counts()

In [ ]:
crashes_cleaned['lighting_condition'].value_counts()

In [ ]:
crashes_cleaned['first_crash_type'].value_counts()

This seems like it could be useful. 

In [ ]:
crashes_cleaned['trafficway_type'].value_counts()

This is important will keep this

In [ ]:
# Define intersection types
intersection_types = ['roundabout', 'l-intersection', 'y-intersection', 
                      'five point, or more', 'center turn lane', 
                      't-intersection', 'unknown intersection type']

# Define conditions for both blocks (with block 2 modification)
conditions = [
    (crashes_cleaned['trafficway_type'] == 'one-way') & (crashes_cleaned['lane_cnt'] == 1),
    (crashes_cleaned['trafficway_type'] == 'one-way') & (crashes_cleaned['lane_cnt'] > 1),
    (crashes_cleaned['trafficway_type'].isin(intersection_types)),
    (crashes_cleaned['trafficway_type'].isin(['unknown', 'not reported'])) | 
    (pd.isnull(crashes_cleaned['trafficway_type'])) | 
    (pd.isnull(crashes_cleaned['lane_cnt'])),
    (crashes_cleaned['trafficway_type'].isin(['parking lot', 'driveway', 'ramp', 'alley', 'other'])),
    # Modified condition for 'multi-lane bidirectional' from Block 2
    (crashes_cleaned['lane_cnt'] > 1) & 
    (~crashes_cleaned['trafficway_type'].isin([
        'one-way', 'four way', 'unknown', 'not reported', 
        'other', 'parking lot', 'driveway', 'ramp', 'alley'
    ]))
]

# Define corresponding categories
choices = [
    'single-lane one way',
    'multi-lane one way',
    'intersection',
    'unknown',  # Combined "unknown" and "not reported"
    'other',
    'multi-lane bidirectional'
]

# Apply classification
crashes_cleaned['road_category'] = np.select(conditions, choices, default='unknown')

In [ ]:
# Check the distribution of categories in the new column
crashes_cleaned['road_category'].value_counts()

In [ ]:
crashes_cleaned['alignment'].value_counts()

While this feature would ideally be helpful in analysis, the data here is not conducive for analysis. Most of the entries are 'straight and level'. will remove.

In [ ]:
crashes_cleaned['roadway_surface_cond'].value_counts()

This is somewhat redundant with weather condition. Will remove weather_condition  and keep roadway_surface_cond due to its lower cardinality

In [ ]:
crashes_cleaned['street_direction'].value_counts()

In [ ]:
crashes_cleaned['street_name'].value_counts()

In [ ]:
crashes_cleaned['road_defect'].value_counts()

The main two categories here are "no defects" and unknown. This will not be helpful for analysis. Will remove

In [ ]:
crashes_cleaned['crash_type'].value_counts()

This describes the aftermath, not helpful for contributory factors. Will remove

In [ ]:
crashes_cleaned['beat_of_occurrence'].value_counts()

High cardinality, unlikely to be useful for analysis. to remove

In [ ]:
crashes_cleaned['most_severe_injury'].value_counts()

In [ ]:
crashes_cleaned['injuries_total'].value_counts()

In [ ]:
crashes_cleaned['injuries_fatal'].value_counts()

In [ ]:
crashes_cleaned['injuries_incapacitating'].value_counts()

The 'injuries_...' features are redundant. This information is captured in the 'most_severe_injury' feature. Will remove all 'injuries_...' features and keep most_severe_injury. 

Most_sever_injury will require cardinality reduction. 

In [ ]:
# Replace the string 'nan' with actual NaN values
crashes_cleaned['most_severe_injury'] = crashes_cleaned['most_severe_injury'].replace('nan', np.nan)

# Now categorize the injuries into 'Serious' and 'Non-serious'
crashes_cleaned['severity_category'] = crashes_cleaned['most_severe_injury'].replace({
    'no indication of injury': 'Non-serious',
    'nonincapacitating injury': 'Non-serious',
    'reported, not evident': 'Non-serious',
    'incapacitating injury': 'Serious',
    'fatal': 'Serious'
})

In [ ]:
crashes_cleaned['prim_contributory_cause'].value_counts()

In [ ]:
crashes_cleaned['sec_contributory_cause'].value_counts()

This feature is redundant to prim_contributory_cause. A high majority of values are either not applicable or unable to determine so it will be dropped. 

In [ ]:
# Create a mapping for the primary contributory causes
cause_mapping = {
    'distraction - from inside vehicle': 'Distraction',
    'distraction - from outside vehicle': 'Distraction',
    'cell phone use other than texting': 'Distraction',
    'distraction - other electronic device (navigation device, dvd player, etc.)': 'Distraction',
    'texting': 'Distraction',
    'bicycle advancing legally on red light': 'Distraction',
    'motorcycle advancing legally on red light': 'Distraction',
    
    'operating vehicle in erratic, reckless, careless, negligent or aggressive manner': 'Aggressive/Reckless Driving',
    'failing to reduce speed to avoid crash': 'Aggressive/Reckless Driving',
    'exceeding authorized speed limit': 'Aggressive/Reckless Driving',
    'exceeding safe speed for conditions': 'Aggressive/Reckless Driving',
    'driving on wrong side/wrong way': 'Aggressive/Reckless Driving',
    'disregarding stop sign': 'Aggressive/Reckless Driving',
    'disregarding traffic signals': 'Aggressive/Reckless Driving',
    'disregarding yield sign': 'Aggressive/Reckless Driving',
    'passing stopped school bus': 'Aggressive/Reckless Driving',
    'improper overtaking/passing': 'Aggressive/Reckless Driving',
    'failing to yield right-of-way': 'Aggressive/Reckless Driving',
    'following too closely': 'Aggressive/Reckless Driving',
    'improper lane usage': 'Aggressive/Reckless Driving',
    'improper turning/no signal': 'Aggressive/Reckless Driving',
    
    'driving skills/knowledge/experience': 'Driver\'s Condition/Experience',
    'physical condition of driver': 'Driver\'s Condition/Experience',
    'vision obscured (signs, tree limbs, buildings, etc.)': 'Driver\'s Condition/Experience',
    'under the influence of alcohol/drugs (use when arrest is effected)': 'Driver\'s Condition/Experience',
    'had been drinking (use when arrest is not made)': 'Driver\'s Condition/Experience',
    
    'weather': 'Environmental and Road Conditions',
    'road engineering/surface/marking defects': 'Environmental and Road Conditions',
    'road construction/maintenance': 'Environmental and Road Conditions',
    'evasive action due to animal, object, nonmotorist': 'Environmental and Road Conditions',
    'animal': 'Environmental and Road Conditions',
    
    'unable to determine': 'Unknown/Other',
    'not applicable': 'Unknown/Other',
    'related to bus stop': 'Unknown/Other',
    'obstructed crosswalks': 'Unknown/Other',
    
    # Add the missing categories
    'improper backing': 'Aggressive/Reckless Driving',
    'equipment - vehicle condition': 'Driver\'s Condition/Experience',
    'disregarding other traffic signs': 'Aggressive/Reckless Driving',
    'disregarding road markings': 'Aggressive/Reckless Driving',
    'turning right on red': 'Aggressive/Reckless Driving'
}

# Apply the mapping to categorize the causes
crashes_cleaned['crash_cause_category'] = crashes_cleaned['prim_contributory_cause'].map(cause_mapping)

In [ ]:
# Find unique values in 'prim_contributory_cause' that are not in the 'cause_mapping'
missing_values = crashes_cleaned[~crashes_cleaned['prim_contributory_cause'].isin(cause_mapping.keys())]['prim_contributory_cause'].unique()

print(missing_values)

In [ ]:
# Check the value counts in the new category column
crashes_cleaned['crash_cause_category'].value_counts()

Damage, 'date_police_notified' both deal with aftermath, Not contributory factors. To be removed

'street_no'... unhelpful. Will remove

'num_units' not helpful in contributory factor, to remove

remove latitude and longitude as these are captured in location feature. 

In [ ]:
crashes_cleaned['crash_date'].value_counts()

Will remove this feature. This information is captured in crash_hour, crash_day_of_the_week, crash_month

In [ ]:
crashes_cleaned['crash_hour'].value_counts()

In [ ]:
crashes_cleaned['crash_day_of_week'].value_counts()

In [ ]:
crashes_cleaned['crash_month'].value_counts()

In [ ]:
crashes_cleaned['time_of_day'] = pd.cut(
    crashes_cleaned['crash_hour'], 
    bins=[-1, 5, 11, 17, 23], 
    labels=['Night (Late)', 'Morning', 'Afternoon', 'Night (Early)'],
    right=True
)

In [ ]:
crashes_cleaned['day_of_week'] = crashes_cleaned['crash_day_of_week'].replace({
    1: 'Sun',
    2: 'Mon',
    3: 'Tues',
    4: 'Wed',
    5: 'Thur',
    6: 'Fri',
    7: 'Sat'
})

In [ ]:
crashes_cleaned['day_of_week'].value_counts()

In [ ]:
crashes_cleaned['season'] = pd.cut(
    crashes_cleaned['crash_month'], 
    bins=[0, 2, 5, 8, 11, 12], 
    labels=['Winter', 'Spring', 'Summer', 'Fall', 'Winter'],
    right=True,
    ordered=False
)

In [ ]:
crashes_cleaned['season'].value_counts()

In [ ]:
crashes_cleaned.info()

In [ ]:
crashes_cleaned.isna().sum()

In [ ]:
crashes_cleaned.drop(columns = [
    'crash_date',
    'hit_and_run_i',
    'device_condition',
    'weather_condition',
    'road_defect',
    'crash_type',
    'damage',
    'date_police_notified',
    'sec_contributory_cause',
    'street_no',
    'report_type',
    'beat_of_occurrence',
    'num_units',
    'alignment',
    'injuries_total',
    'injuries_fatal',
     'injuries_incapacitating',
     'injuries_non_incapacitating',
     'injuries_reported_not_evident',
     'injuries_no_indication',
    'injuries_unknown',
    'location',
    'street_direction',
    'lane_cnt', 
    'intersection_related_i',
    'trafficway_type', 
    'crash_hour', 
    'crash_day_of_week', 
    'crash_month', 
    'posted_speed_limit', 
    'traffic_control_device', 
    'street_name', 
    'most_severe_injury',
    'prim_contributory_cause',
    'latitude',
    'longitude'
], inplace = True)

In [ ]:
crashes_cleaned.info()

In [ ]:
# Convert all the columns (except 'crash_record_id') to category type
crashes_cleaned[[col for col in crashes_cleaned.columns if col != 'crash_record_id']] = crashes_cleaned[[col for col in crashes_cleaned.columns if col != 'crash_record_id']].astype('category')

# Verify the changes
crashes_cleaned.dtypes

In [ ]:
(crashes_cleaned.isna().sum()/ len(crashes_cleaned))* 100

In [ ]:
crashes_cleaned['severity_category'].value_counts()

In [ ]:
crashes_cleaned.dropna(inplace = True)

In [ ]:
crashes_cleaned.isna().sum()

In [ ]:
crashes_cleaned['severity_category'].value_counts()

In [ ]:
list(crashes_cleaned.columns)

### 2.2 People 

2.2.1 Preview data

2.2.2 Understand Structure

2.2.3 Format Feature names and Row Values

2.2.4 Drop features with overly high null values

2.2.5 Checking for duplicates

2.2.6 Inspect remaining features

2.2.7 remove unuseful feature

2.2.8 convert data types

2.2.9 reduce feature cardinality with bucketing

2.2.10 remove remaining nulls

2.2.11 Merge Preparation: Aggregation

Steps:

* Steps:
* **Preview Data**: `.head()`


* **Understand Structure**: `.info()`


* **Format Feature names and Row Values**: `.lower()`


* **Drop features with overly high null values**: `.isna().sum()/ len(df)` for percentage of nulls for each feature


* **Check for duplicates**: `.duplicated().sum()`


* **inspect remaining features**: `.value_counts()`; 

    * make intentional decisions to keep or drop using `.value_counts()` distribution and domain knowledge; 
    * make note of any features to keep that will need cleaning/cardinality reduction/etc.


* **remove unuseful features**: 

    * `.drop()` for list of features deemed not useful for analysis; 
    * store trimmed df as 'df_name_cleaned'


* **Convert data types**: 

    * stored data types to reflect true data types 
    * (categorical variables as strings, numeric variables as int, ect.)


* **reduce feature cardinality with bucketing**:

    * 'safety_equipment' to 'safety_equipment_category'
    * 'age' to 'age_group'
 
 
* **remove remaining nulls**: `.dropna()`


* **Merge Preparation: Aggregation**

In [ ]:
people = pd.read_csv('./data/people.csv', low_memory = True)

In [ ]:
people.head()

In [ ]:
people.info()

In [ ]:
people.columns = people.columns.str.lower()

In [ ]:
# Convert all string values in object columns to lowercase
for col in people.select_dtypes(include='object').columns:
    people[col] = people[col].str.lower()

In [ ]:
round((people.isna().sum()/ len(people)*100), 2)

In [ ]:
# selecting all features with 90% or more of its values are null
ppl_high_null_features = people.columns[(people.isna().sum() / len(people) * 100) >= 90]
ppl_high_null_features

In [ ]:
# creating a list of features with 90% or more null values
ppl_high_null_features_list = list(ppl_high_null_features)
ppl_high_null_features_list

In [ ]:
people_cleaned = people.drop(columns=ppl_high_null_features_list)
people_cleaned.info()

#### Checking for duplicates

In [ ]:
# Define the list of ID columns to check for duplicates
id_columns = ['person_id', 'crash_record_id', 'vehicle_id']

# Loop through each column to check for duplicates
for column in id_columns:
    
    # Check for duplicates in the current column
    duplicates = people_cleaned.duplicated(subset=[column])
    
    # Get the count of duplicate rows
    duplicates_count = duplicates.sum()
    
    # Print the count of duplicates
    print(f"Number of duplicate {column} rows: {duplicates_count}")
    
    # Optionally, view the actual duplicate rows for the current column
    duplicate_rows = people_cleaned[duplicates]
    print(f"\n{column} duplicate rows:")
    print(duplicate_rows)
    print("\n" + "="*50)  # Separator for readability

##### Explanation of Output:

1. **Duplicate person_id Rows:**
   - No duplicate person_id rows were found, which means each person_id is unique in this dataset.

2. **Duplicate crash_record_id Rows:**
   - A total of 1,080,392 rows are duplicates based on crash_record_id. This suggests that multiple individuals (drivers, passengers, etc.) may have been associated with the same crash. This is expected if there are multiple people involved in the same crash event.

3. **Duplicate vehicle_id Rows:**
   - A total of 421,011 rows are duplicates based on vehicle_id. This indicates that some vehicles appear in multiple records, potentially due to different passengers or crashes involving the same vehicle.

In [ ]:
people_cleaned[people_cleaned.duplicated(keep=False)]

In [ ]:
people_cleaned['person_type'].value_counts()

In [ ]:
people_cleaned['crash_date'].value_counts()

In [ ]:
people_cleaned['seat_no'].value_counts()

In [ ]:
people_cleaned['safety_equipment'].value_counts(normalize = True)

Could be an important predictor, but will need to reduce cardinality. 

In [ ]:
# Create a dictionary to map the original 'safety_equipment' values to broader categories
safety_equipment_mapping = {
    # Used Equipment
    'safety belt used': 'Used',
    'child restraint used': 'Used',
    'child restraint - forward facing': 'Used',
    'bicycle helmet (pedacyclist involved only)': 'Used',
    'child restraint - type unknown': 'Used',
    'child restraint - rear facing': 'Used',
    'dot compliant motorcycle helmet': 'Used',
    'helmet used': 'Used',
    'booster seat': 'Used',
    'child restraint used improperly': 'Used',

    # Not Used Equipment
    'safety belt not used': 'Not Used',
    'helmet not used': 'Not Used',
    'child restraint not used': 'Not Used',
    'not dot compliant motorcycle helmet': 'Not Used',
    'should/lap belt used improperly': 'Not Used',

    # Unknown Equipment Usage
    'usage unknown': 'Unknown',

    # Other/Special Case Equipment
    'none present': 'Other/Special Case', 
    'wheelchair': 'Other/Special Case',
    'stretcher': 'Other/Special Case',
    'unknown': 'Other/Special Case',  # Catch-all for any unknown or missing values
}

# Apply the mapping to the 'safety_equipment' column
people_cleaned['safety_equipment_category'] = people_cleaned['safety_equipment'].map(safety_equipment_mapping)

In [ ]:
# Check the value counts for the new grouped categories
people_cleaned['safety_equipment_category'].value_counts(normalize = True)

Based on this output, even after recategorization, this feature will not be very useful. About 95% of the data is split between "used" and "unknown", with the remaining 5% split between "other/special case" and "not used"

In [ ]:
people_cleaned['airbag_deployed'].value_counts()

This feature might be more helpful if we simply knew: did the airbag deploy or not?

In [ ]:
# Define the mapping for airbag_deployed
airbag_mapping = {
    'did not deploy': 'Not Deployed',
    'not applicable': 'Not Deployed',  # Assuming "not applicable" should be considered as unknown
    'deployment unknown': 'Unknown',
    'deployed, front': 'Deployed',
    'deployed, combination': 'Deployed',
    'deployed, side': 'Deployed',
    'deployed other (knee, air, belt, etc.)': 'Deployed'
}

# Apply the mapping to the 'airbag_deployed' column
people_cleaned['airbag_deployed'] = people_cleaned['airbag_deployed'].map(airbag_mapping)

In [ ]:
# Optionally, check the new value counts
people_cleaned['airbag_deployed'].value_counts()

In [ ]:
people_cleaned['ejection'].value_counts()

This feature might be normally be helpful, but it is far too skewed to be helpful for this analysis. Only about 8.5k values other than 'none' or 'unknown'. This will be removed

In [ ]:
people_cleaned['injury_classification'].value_counts()

We can drop this. It contains similar information to 'most_severe_injury' but is more imbalanced so I will drop it and keep 'most_severe_injury' as my target. 

In [ ]:
round(people_cleaned['driver_action'].value_counts(normalize = True), 2)

similar to 'prim_contributory_cause' in crashes. but this one contains 20% nulls. Will drop this and keep prim_contributory_cause.

In [ ]:
people_cleaned['driver_vision'].value_counts()

This feature is too skewed to provide any real analytical benefit. The top two by 500k values are 'not_obscured' and 'unknown'

In [ ]:
people_cleaned['physical_condition'].value_counts()

Most are normal or unknown. Not particularly helpful. Remove

In [ ]:
people_cleaned['bac_result'].value_counts()

most are test not offered and test refused. Again, not very helpful, so will drop

In [ ]:
people_cleaned['age'].value_counts()

Could be interesting to investigate, but high cardinality means it will need some bucketing

In [ ]:
# Sample data (replace with your actual DataFrame)
age_bins_df = pd.DataFrame({
    'age': [5, 15, 16, 25, 30, 60, 100, 120, -5, 200]
})

# Define the bins for age groups
age_bins = [1, 16, 27, 66, 115]

# Labels for the age groups
age_labels = ['1-15', '16-26', '27-65', '65+']  

# Apply corrections for age values outside the valid range (negative, 0, or greater than 115)
people_cleaned['age'] = people_cleaned['age'].apply(lambda x: np.nan if x < 1 or x > 115 else x)

# Apply pd.cut() to create a new 'age_group' column
people_cleaned['age_group'] = pd.cut(people_cleaned['age'], bins=age_bins, labels=age_labels, right=False)

# Print the first few rows to verify the new grouping
print(people_cleaned[['age', 'age_group']].head())

Drivers license state and class is not relevant. 

In [ ]:
people_cleaned.drop(columns=['person_id',
                      'person_type',
                      'vehicle_id',
                      'drivers_license_state', 
                      'drivers_license_class',
                      'city', 
                      'state', 
                      'zipcode',
                      'hospital', 
                      'crash_date',
                      'seat_no',
                      'ejection',
                      'injury_classification',
                      'driver_vision',
                      'driver_action',
                      'physical_condition',
                      'bac_result',
                      'age', 
                      'safety_equipment',
                      'safety_equipment_category'
                     ], inplace = True)
people_cleaned.info()

In [ ]:
for feature in people_cleaned.columns:
    print(people_cleaned[feature].value_counts())
    print()
    print("-" * 32)

In [ ]:
# Convert the specified columns to string type
people_cleaned[['age_group', 'airbag_deployed', 'sex']] = people_cleaned[['age_group', 'airbag_deployed', 'sex']].astype('category')

In [ ]:
people_cleaned.isna().sum()

In [ ]:
len(people_cleaned)

In [ ]:
people_cleaned.dropna(inplace = True)

In [ ]:
people_cleaned.info()

In [ ]:
people[['age', 'sex', 'airbag_deployed']].isna().sum()

In [ ]:
people_cleaned.isna().sum()

The goal of aggregating the people_cleaned dataset is to handle the many-to-one relationship between the people_cleaned and crashes_cleaned datasets. Each crash_record_id in crashes_cleaned may have multiple associated records in people_cleaned, as a single crash may involve multiple people. Since our focus is on predicting the severity of crashes using the severity_category (the target variable in crashes_cleaned), we need to aggregate the people data to ensure each crash record has only one corresponding row of data.

The aggregation process involves grouping the people_cleaned dataset by crash_record_id, which is the shared key between the two datasets. For features like sex, age_group, and airbag_deployed, we use the most frequent value for each crash. In cases where there is a tie (e.g., multiple values with the same frequency), we resolve the tie by selecting the value with the highest count using the idxmax() function on the value counts of each group. This ensures consistency and avoids ambiguity in cases of a tie.

In [ ]:
def resolve_tie(x):
    """
    Resolves ties in categorical data by selecting the most frequent value.
    If the series is empty or contains only NaNs, returns NaN.
    """
    if x.isna().all():
        return np.nan  # or another placeholder value
    return x.value_counts().idxmax()  # Pick the most frequent value in case of a tie

In [ ]:
# Aggregating the people data by crash_record_id

# First, optimize tie resolution and avoid unnecessary grouping
sex_aggregated = people_cleaned.groupby('crash_record_id')['sex'].apply(resolve_tie).reset_index()
age_group_aggregated = people_cleaned.groupby('crash_record_id')['age_group'].apply(lambda x: x.mode().iloc[0]).reset_index()
airbag_aggregated = people_cleaned.groupby('crash_record_id')['airbag_deployed'].apply(resolve_tie).reset_index()

# Merge the results
people_aggregated = sex_aggregated.merge(age_group_aggregated, on='crash_record_id').merge(airbag_aggregated, on='crash_record_id')

# Preview the aggregated people data
people_aggregated.head()

### 2.3 Vehicles

2.3.1 Preview Data

2.3.2 Understand Structure

2.3.3 Format Feature names and Row Values

2.3.4 Drop features with overly high null values

2.3.5 Check for duplicates

2.3.6 inspect remaining features


2.3.6.1 reduce feature cardinality with bucketing

2.3.7 remove unuseful features

2.3.8 Convert data types

2.3.9 remove remaining nulls

2.3.10 Merge Preparation: Aggregation

Steps:

* Steps:
* **Preview Data**: `.head()`


* **Understand Structure**: `.info()`


* **Format Feature names and Row Values**: `.lower()`


* **Drop features with overly high null values**: `.isna().sum()/ len(df)` for percentage of nulls for each feature


* **Check for duplicates**: `.duplicated().sum()`


* **inspect remaining features**: `.value_counts()`; 

    * make intentional decisions to keep or drop using `.value_counts()` distribution and domain knowledge; 
    * make note of any features to keep that will need cleaning/cardinality reduction/etc.


* **remove unuseful features**: 

    * `.drop()` for list of features deemed not useful for analysis; 
    * store trimmed df as 'df_name_cleaned'


* **reduce feature cardinality with bucketing**:

    * 'safety_equipment' to 'safety_equipment_category'
    * 'age' to 'age_group'
 
 
* **Convert data types**: 

    * stored data types to reflect true data types 
    * (text as string, categorical variables as categories, numeric variables as int, ect.) 
 
* **remove remaining nulls**: `.dropna()`


* **Merge Preparation: Aggregation**

In [ ]:
vehicles = pd.read_csv('./data/vehicles.csv', low_memory = True)
vehicles.head()

In [ ]:
vehicles.info()

In [ ]:
vehicles.columns = vehicles.columns.str.lower()

In [ ]:
# Convert all string values in object columns to lowercase
for col in vehicles.select_dtypes(include='object').columns:
    vehicles[col] = vehicles[col].str.lower()

In [ ]:
# selecting all features with 90% or more of its values are null
high_null_features = vehicles.columns[(vehicles.isna().sum() / len(vehicles) * 100) >= 90]
high_null_features

In [ ]:
# creating a list of features with 90% or more null values
high_null_features_list = list(high_null_features)

In [ ]:
vehicles_cleaned = vehicles.drop(columns=high_null_features_list)

In [ ]:
# Check for duplicates based on crash_unit_id, crash_record_id, and vehicle_id
vehicles_cleaned[vehicles_cleaned.duplicated(subset=['crash_unit_id', 'crash_record_id', 'vehicle_id'], keep=False)]


The output tells us that there are no duplicate rows in the vehicles_cleaned DataFrame based on the specified subset of columns: crash_unit_id, crash_record_id, and vehicle_id.

In [ ]:
vehicles_cleaned['num_passengers'].value_counts()

This is redundant information. This information does not include the driver, but this information is captured in occupant count. will drop this one. 

In [ ]:
vehicles_cleaned['unit_no'].value_counts()

This is aftermath. unhelpful. remove

In [ ]:
vehicles_cleaned['unit_type'].value_counts()

Most of the values are drivers or parked cars. this will not be useful for analysis

In [ ]:
vehicles_cleaned['make'].value_counts()

high cardinality

In [ ]:
vehicles_cleaned['model'].value_counts()

This feels like it could be helpful, but many unknowns and 'other', and very high cardinality. The important information that we'd gain from this is already included in vehicle_type. So we can drop

In [ ]:
vehicles_cleaned['vehicle_defect'].value_counts()

Most of the values are none or unknown. This will not be particularly useful for analysis. can drop

In [ ]:
vehicles_cleaned['vehicle_type'].value_counts()

In [ ]:
vehicles_cleaned['travel_direction'].value_counts()

Unhelpful for analysis. Remove

In [ ]:
vehicles_cleaned['maneuver'].value_counts()

This feature could be important as it has to do with what was happening prior to the crash.

In [ ]:
vehicles_cleaned['towed_i'].value_counts()

Aftermath; Unhelpful for analysis

In [ ]:
vehicles_cleaned['occupant_cnt'].value_counts()

It is unclear what the area_##_i features represent. They will be removed

In [ ]:
vehicles_cleaned['first_contact_point'].value_counts()

This feature could indicate

In [ ]:
vehicle_features_to_drop = ['num_passengers', 
                            'crash_unit_id',
                            'crash_date',
                            'unit_type',
                            'make', 
                            'model',
                            'vehicle_id',
                           'vehicle_defect',
                           'unit_no',
                           'lic_plate_state',
                            'vehicle_year',
                           'vehicle_use',
                           'travel_direction',
                           'towed_i',
                            'area_01_i',
                           'area_02_i', 
                            'area_05_i',
                            'area_06_i',
                            'area_07_i',
                            'area_08_i',
                            'area_10_i',
                            'area_11_i',
                            'area_12_i',
                            'area_99_i', 
                           'first_contact_point']

In [ ]:
vehicles_cleaned = vehicles_cleaned.drop(columns=vehicle_features_to_drop)

In [ ]:
vehicles_cleaned.info()

In [ ]:
list(vehicles_cleaned.columns)

In [ ]:
# Create a dictionary to map the original vehicle types to more specific categories
vehicle_type_mapping = {
    'passenger': 'Passenger Vehicles',
    'sport utility vehicle (suv)': 'SUVs',
    'van/mini-van': 'Passenger Vehicles',
    'pickup': 'Trucks',
    'truck - single unit': 'Trucks',
    'single unit truck with trailer': 'Trucks',
    'other': 'Other',
    'bus over 15 pass.': 'Buses',
    'bus up to 15 pass.': 'Buses',
    'tractor w/ semi-trailer': 'Trucks',
    'tractor w/o semi-trailer': 'Trucks',
    'motorcycle (over 150cc)': 'Motorcycles',
    'other vehicle with trailer': 'Other',
    'autocycle': 'Motorcycles',
    'moped or motorized bicycle': 'Motorcycles',
    'motor driven cycle': 'Motorcycles',
    'all-terrain vehicle (atv)': 'Recreational/Off-Highway Vehicles',
    'farm equipment': 'Farm and Specialized Equipment',
    '3-wheeled motorcycle (2 rear wheels)': 'Motorcycles',
    'recreational off-highway vehicle (rov)': 'Recreational/Off-Highway Vehicles',
    'snowmobile': 'Recreational/Off-Highway Vehicles',
    'unknown/na': np.nan  # Set 'unknown/na' to NaN
}

# Apply the mapping to the 'vehicle_type' column
vehicles_cleaned['vehicle_category'] = vehicles_cleaned['vehicle_type'].map(vehicle_type_mapping)

# Check the value counts for the new grouped categories
vehicles_cleaned['vehicle_category'].value_counts()

In [ ]:
# Filter out rows with 'Recreational/Off-Highway Vehicles' and 'Farm and Specialized Equipment'
vehicles_cleaned = vehicles_cleaned[~vehicles_cleaned['vehicle_category'].isin(['Recreational/Off-Highway Vehicles', 'Farm and Specialized Equipment'])]

# Check the value counts after removing those categories
vehicles_cleaned['vehicle_category'].value_counts()


In [ ]:
vehicles_cleaned.info()

In [ ]:
# Modify the maneuver mapping to treat 'unknown/na' as NaN
maneuver_mapping = {
    'straight ahead': 'Standard Movement',
    'slow/stop in traffic': 'Standard Movement',
    'passing/overtaking': 'Standard Movement',
    'unknown/na': np.nan,  # Set 'unknown/na' to NaN
    
    'parked': 'Reversing/Stopping',
    'entering traffic lane from parking': 'Reversing/Stopping',
    'starting in traffic': 'Reversing/Stopping',
    
    'turning left': 'Turn/Change of Direction',
    'turning right': 'Turn/Change of Direction',
    'u-turn': 'Turn/Change of Direction',
    'changing lanes': 'Turn/Change of Direction',
    'turning on red': 'Turn/Change of Direction',
    
    'backing': 'Reversing/Stopping',
    'avoiding vehicles/objects': 'Avoidance/Emergency Response',
    'skidding/control loss': 'Avoidance/Emergency Response',
    'negotiating a curve': 'Avoidance/Emergency Response',
    
    'leaving traffic lane to park': 'Reversing/Stopping',
    'enter from drive/alley': 'Reversing/Stopping',
    
    'driving wrong way': 'Special Cases',
    'diverging': 'Special Cases',
    'driverless': 'Special Cases',
    'disabled': 'Special Cases',
    
    'other': 'Special Cases',
}

# Apply the mapping to the 'maneuver' column
vehicles_cleaned['maneuver_category'] = vehicles_cleaned['maneuver'].map(maneuver_mapping)

# Check the value counts for the new 'maneuver_category'
vehicles_cleaned['maneuver_category'].value_counts()

In [ ]:
# Replace 0, 99, and negative values with NaN in 'occupant_cnt' for future dropping
vehicles_cleaned['occupant_cnt'] = vehicles_cleaned['occupant_cnt'].replace([0, 99], np.nan)

# Define bins for the categories (including the 20-98 range for Very Large Group)
bins = [1, 4, 8, 19, 98, float('inf')]  # Adjusted to include 20-98 for Very Large Group
labels = ['Single Occupancy', 'Small Group', 'Medium Group', 'Large Group', 'Very Large Group']  # 5 labels for 5 bins

# Use pd.cut to categorize the 'occupant_cnt' column based on the bins
vehicles_cleaned['occupant_category'] = pd.cut(
    vehicles_cleaned['occupant_cnt'], 
    bins=bins, 
    labels=labels, 
    right=True, 
    include_lowest=False  # Exclude 0 from Single Occupancy
)

# Handle the NaN values for 'occupant_category' (those rows with invalid occupant_cnt, such as 0 or 99)
vehicles_cleaned['occupant_category'] = vehicles_cleaned['occupant_category'].where(
    vehicles_cleaned['occupant_category'].notna(), np.nan
)

# Check the categories to ensure the correct bucketing
vehicles_cleaned['occupant_category'].value_counts()

In [ ]:
# Remove 'Very Large Group' rows since it contains zero values
vehicles_cleaned = vehicles_cleaned[vehicles_cleaned['occupant_category'] != 'Very Large Group']

# Drop 'Very Large Group' from the categorical data if it still exists
vehicles_cleaned['occupant_category'] = vehicles_cleaned['occupant_category'].cat.remove_categories('Very Large Group')

# Check the categories to ensure the correct bucketing
vehicles_cleaned['occupant_category'].value_counts()

In [ ]:
vehicles_cleaned.info()

In [ ]:
vehicles_cleaned.isna().sum()

In [ ]:
vehicles['occupant_cnt'].isna().sum()

In [ ]:
vehicles['occupant_cnt'].value_counts()

In [ ]:
# Drop the 'occupant_category' and 'occupant_cnt' column
vehicles_cleaned = vehicles_cleaned.drop(columns=['occupant_category', 'occupant_cnt', 'vehicle_type', 'maneuver'])


In [ ]:
vehicles_cleaned.isna().sum()

In [ ]:
# Drop rows with any null (NaN) values
vehicles_cleaned = vehicles_cleaned.dropna()

In [ ]:
vehicles_cleaned.info()

In [ ]:
vehicles_cleaned.isna().sum()

Similarly, the vehicles_cleaned dataset also has a many-to-one relationship with crashes_cleaned, where each crash_record_id in crashes_cleaned can have multiple associated vehicle records. As with the people_cleaned data, we need to aggregate the vehicle data to ensure that each crash record has a corresponding single row. The aggregation will allow us to focus on features like vehicle_category and other vehicle-specific attributes that might affect crash severity.

By grouping the vehicles_cleaned dataset by crash_record_id, we can apply the same aggregation logic as with the people data. This ensures that we retain the most important vehicle-specific features while also maintaining a consistent one-to-one relationship between crashes_cleaned and the aggregated datasets.

In [ ]:
# Aggregating the vehicles data by crash_record_id
vehicles_aggregated = vehicles_cleaned.groupby('crash_record_id').agg({
    'vehicle_category': resolve_tie,  # Resolve tie by choosing the most frequent vehicle category
    'maneuver_category': resolve_tie,  # Resolve tie by choosing the most frequent maneuver category
}).reset_index()

# View the aggregated vehicles data
vehicles_aggregated.head()

### Merging `crashes_cleaned`, `people_cleaned`, & `vehicles_cleaned`

Relationships Between Tables and Justification for Merging

1. Relationship Between crashes_cleaned and people_cleaned
	•	crash_record_id is the primary key in crashes_cleaned and appears in people_cleaned.
	•	Each crash in crashes_cleaned can involve multiple people (drivers, passengers, pedestrians).

This means the relationship is:
	•	One-to-Many: One crash (crashes_cleaned) can have many people (people_cleaned) associated with it.

2. Relationship Between crashes_cleaned and vehicles_cleaned
	•	crash_record_id is the primary key in crashes_cleaned and appears in vehicles_cleaned.
	•	Each crash in crashes_cleaned can involve multiple vehicles.

This means the relationship is:
	•	One-to-Many: One crash (crashes_cleaned) can have many vehicles (vehicles_cleaned) associated with it.

3. Relationship Between people_cleaned and vehicles_cleaned
	•	Both tables are linked via crash_record_id, but they describe different entities.
	•	People (people_cleaned) and vehicles (vehicles_cleaned) may not have a direct relationship unless there’s another shared identifier (e.g., vehicle_id or person_id).

This means the relationship is:
	•	Many-to-Many: Many people can be in many vehicles within the same crash. (However, this relationship is indirectly expressed through crash_record_id.)

In [ ]:
# Merge the dataframes using an inner join
merged_df = crashes_cleaned.merge(people_aggregated, on='crash_record_id', how='inner')

In [ ]:
merged_df = merged_df.merge(vehicles_aggregated, on='crash_record_id', how='inner')

In [ ]:
merged_df.info()

In [ ]:
# Convert all the columns (except 'crash_record_id') to category type
merged_df[[col for col in merged_df.columns if col != 'crash_record_id']] = merged_df[[col for col in merged_df.columns if col != 'crash_record_id']].astype('category')

In [ ]:
# Verify the changes
merged_df.dtypes

In [ ]:
crashes.columns = crashes.columns.str.lower()

In [ ]:
# Convert all string values in object columns to lowercase
for col in crashes.select_dtypes(include='object').columns:
    crashes[col] = crashes[col].str.lower()

In [ ]:
# Check the result
merged_df.isna().sum()

In [ ]:
merged_df['severity_category'].value_counts()

In [ ]:
# Check for duplicate crash_record_id values
merged_df[merged_df.duplicated(subset='crash_record_id', keep=False)]

In [ ]:
# Iterate over each column in the DataFrame and print value counts for each feature
for column in merged_df.columns:
    print(f"Value counts for {column}:")
    print(merged_df[column].value_counts())
    print("-" * 50)  # Optional separator for readability

In [ ]:
merged_df.isna().sum()

In [ ]:
# List of columns you want to clean
columns_to_clean = ['airbag_deployed', 'roadway_surface_cond', 'lighting_condition']  # replace with your actual column names

# Iterate over each column in the list
for column in columns_to_clean:
    # Check for unwanted values in the current column and remove rows
    merged_df = merged_df[~merged_df[column].str.contains(
        'unknown|not applicable|other object|unknown/other', case=False, na=False)]

# Verify the changes
merged_df.info()

In [ ]:
# Check the DataFrame after removal
merged_df.info()

In [ ]:
merged_df['severity_category'].value_counts(normalize = True)

From the above output, we see that the classes are greatly imbalanced. This will be something I will need to address during the modeling phase

In [ ]:
# grouping classes into two groups for binary classification: 0 and 1
merged_df.severity_category.replace({
    'Serious' : 1,
    'Non-serious' : 0},
    inplace = True
)

In [ ]:
# Step 1: Preprocessing the data with OneHotEncoder for categorical features
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


x = merged_df.drop(columns=['severity_category'], axis = 1)  # Drop the target column to get the features
y = merged_df['severity_category']

ohe = OneHotEncoder(sparse=False)  # sparse=False to return an array instead of a sparse matrix

# Fit the encoder and transform the categorical features
x_ohe = ohe.fit_transform(x)

# Convert the transformed data into a DataFrame for easy column naming
ohe_df = pd.DataFrame(x_ohe, columns=ohe.get_feature_names_out(x.columns))

# Step 2: Train a Decision Tree Classifier
mgmt_tree = DecisionTreeClassifier(criterion='entropy', random_state=42)
mgmt_tree.fit(ohe_df, y)

# Step 3: Use feature_importances_ to get the importance of features
feature_importances = mgmt_tree.feature_importances_

# Step 4: Sort and visualize feature importances
sorted_indices = feature_importances.argsort()[::-1]  # Sort in descending order
sorted_feature_names = ohe_df.columns[sorted_indices]
sorted_importances = feature_importances[sorted_indices]

# Create a bar plot of the feature importances
sns.set(rc={'figure.figsize':(11.7, 8.27)})  # Set the plot size
sns.barplot(x=sorted_importances, y=sorted_feature_names)

# Add title and labels
plt.title('Feature Importances from Decision Tree Classifier', fontsize=16)
plt.xlabel('Importance Score', fontsize=14)
plt.ylabel('Feature', fontsize=14)

# Increase tick mark size
plt.tick_params(axis='both', which='major', labelsize=11)

# Save the plot if desired and display it
plt.savefig("./feature_importances.png", dpi=300, bbox_inches='tight')
plt.show()